# Title: Alert Investigation (Process Alerts)
LogAnalytics
Version 0.3
## Description:
Series of modules designed to help get a better understanding of the contents of a process-based alert.


<a id='toc'></a>
## Table of Contents
- [Setup and Authenticate](#setup)

- [Get Alerts List](#getalertslist)
- [Choose an Alert to investigate](#enteralertid)
  - [Extract Properties and entities from alert](#extractalertproperties)
  - [Entity Graph](#entitygraph)
- [Related Alerts](#related_alerts)
- [Session Process Tree](#processtree)
  - [Process Timeline](#processtimeline)
- [Other Process on Host](#process_clustering)
- [Check for IOCs in Commandline](#cmdlineiocs)
  - [VirusTotal lookup](#virustotallookup)
- [Alert command line - Occurrence on other hosts in subscription](#cmdlineonotherhosts)
- [Host Logons](#host_logons)
  - [Alert Account](#logonaccount)
  - [Failed Logons](#failed_logons)
- [Appendices](#appendices)
  - [Saving data to Excel](#appendices)


<a id='setup'></a>[Contents](#toc)
# Setup

1. Make sure that you have installed packages specified in the setup (uncomment the lines to execute)
2. There are some manual steps up to selecting the alert ID. After this most of the notebook can be executed sequentially
3. Major sections should be executable independently (e.g. Alert Command line and Host Logons can be run skipping Session Process Tree)

## Install Packages

In [ ]:
# You may needs these - should only need to uncomment and run once
!pip install git+https://github.com/ianhelle/msticpy --upgrade  --user
!pip install Kqlmagic --no-cache-dir --upgrade --user

# Provisional location

# !pip install msgpack
# !pip install Kqlmagic --no-cache-dir  --upgrade

# !pip install PyHamcrest
# !conda install -c conda-forge python-levenshtein -y
# !conda install requests
# !conda install attrs
# !conda install seaborn
# !conda install bokeh
# !conda install holoviews

!pip uninstall attr --yes

Bugs:
    
!pip uninstall attr --yes
networkx may be installed



### Import Python Packages

In [ ]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


import numpy as np
from IPython import get_ipython
from IPython.display import display, HTML
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
sns.set()
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

 
import msticpy.sectools as sectools
import msticpy.nbtools as mas
import msticpy.nbtools.kql as qry
import msticpy.nbtools.nbdisplay as nbdisp


### Enter or confirm WorkspaceId
To find your Workspace Id go to [Log Analytics](#https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.

In [ ]:
path = %env PATH
dsvm = False
if '/dsvm/' in path:
    dsvm = True
    
if dsvm == False:
    # Run this if you are using Free Compute
    from utils import config_reader as mod
    workspace_id = mod.config_reader.read_config_values("config.json")[3]
    tenant_id = mod.config_reader.read_config_values("config.json")[0]

    ######################################################################
    your_workspace_name = mod.config_reader.read_config_values("config.json")[4]
    print("Your Log Analytic Workspace: " + your_workspace_name)
    ######################################################################
else:
    # Run this if you are using DSVM.  You need to copy the values from config.json, if the file has no value, then you need to go to Log Analytics Portal to get the information.
    tenant_id = input('tenant_id:')
    subscription_id = input('subscription_id:')
    resource_group = input('resource_group:')
    workspace_id = input('workspace_id:')
    workspace_name = input('workspace_name:')
    

### Authenticate to Log Analytics
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

In [ ]:
WORKSPACE_ID = workspace_id
    
if not WORKSPACE_ID:
    raise ValueError('No workspace selected.')

mas.kql.load_kql_magic()

%kql loganalytics://code().workspace(WORKSPACE_ID)

<a id='getalertslist'></a>[Contents](#toc)
# Get Alerts List

Specify a time range to search for alerts. One this is set run the following cell to retrieve any alerts in that time window.
You can change the time range and re-run the queries until you find the alerts that you want.

In [ ]:
alert_q_times = mas.QueryTime(units='day', max_before=20, before=5, max_after=1)
alert_q_times.display()

In [ ]:
alert_counts = qry.list_alerts_counts(provs=[alert_q_times])
alert_list = qry.list_alerts(provs=[alert_q_times])
print(len(alert_counts), ' distinct alert types')
print(len(alert_list), ' distinct alerts')
display(HTML('<h2>Alert Timeline</h2>'))
nbdisp.display_timeline(data=alert_list, source_columns = ['AlertName', 'CompromisedEntity'], title='Alerts', height=200)
display(HTML('<h2>Top alerts</h2>'))
alert_counts.head(20) # remove '.head(20)'' to see the full list grouped by AlertName

<a id='enteralertid'></a>[Contents](#toc)
# Choose Alert to Investigate
Either pick an alert from a list of retrieved alerts or paste the SystemAlertId into the text box in the following section.

### Select alert from list
As you select an alert, the main properties will be shown below the list.

Use the filter box to narrow down your search to any substring in the AlertName.

In [ ]:
alert_select = mas.AlertSelector(alerts=alert_list, action=nbdisp.display_alert)
alert_select.display()

### Or paste in an alert ID and fetch it
**Skip this if you selected from the above list**

In [ ]:
# Allow alert to be selected
# Allow subscription to be selected
get_alert = mas.GetSingleAlert(action=nbdisp.display_alert)
get_alert.display()

<a id='extractalertproperties'></a>[Contents](#toc)
## Extract properties and entities from Alert
This section extracts the alert information and entities into a SecurityAlert object allowing us to query the properties more reliably. 

In particular, we use the alert to automatically provide parameters for queries and UI elements.
Subsequent queries will use properties like the host name and derived properties such as the OS family (Linux or Windows) to adapt the query. Query time selectors like the one above will also default to an origin time that matches the alert selected.

The alert view below shows all of the main properties of the alert plus the extended property dictionary (if any) and JSON representations of the Entity.

In [ ]:
# Extract entities and properties into a SecurityAlert class
if alert_select.selected_alert is None:
    sys.exit("Please select an alert before executing remaining cells.")

security_alert = mas.SecurityAlert(alert_select.selected_alert)
mas.disp.display_alert(security_alert, show_entities=True)

<a id='entitygraph'></a>[Contents](#toc)
## Entity Graph
Depending on the type of alert there may be one or more entities attached as properties. Entities are things like Host, Account, IpAddress, Process, etc. - essentially the 'nouns' of security investigation. Events and alerts are the things that link them in actions so can be thought of as the verbs. Entities are often related to other entities - for example a process will usually have a related file entity (the process image) and an Account entity (the context in which the process was running). Endpoint alerts typically always have a host entity (which could be a physical or virtual machine).

### Plot using Networkx/Matplotlib

In [ ]:
# Draw the graph using Networkx/Matplotlib
%matplotlib inline
alertentity_graph = mas.create_alert_graph(security_alert)
nbdisp.draw_alert_entity_graph(alertentity_graph, width=15)

In [ ]:
# from pyvis.network import Network
# import math
# # import networkx as nx
# # G = Network()
# # G.from_nx(alertentity_graph)
# import holoviews as hv
# hv.extension('bokeh')
# %opts Graph [width=900 height=900]

# %opts Graph [color_index='circle']
# %opts Graph (node_size=20 edge_line_width=1)
# %opts Graph [tools=['wheel_zoom', 'hover']]
# padding = dict(x=(-1.2, 1.2), y=(-1.2, 1.2))

# n_nodes = len(alertentity_graph.nodes)
# k = 1 / (math.sqrt(n_nodes))

# # hv_graph = hv.Graph.from_networkx(nx_graph, nx.layout.spring_layout, k=k).redim.range(**padding)
# hv_graph = hv.Graph.from_networkx(alertentity_graph, nx.layout.spring_layout, k=k).redim.range(**padding)
# labels = hv.Labels(
#         {('x', 'y'): hv_graph.nodes.array([0, 1]), 'text': hv_graph.nodes.data['name']}, ## 'label' can be an array: has to be correct size!
#         ['x', 'y'], 
#         'text').options(fontsize=8, cmap='viridis', yoffset=0.05)
# hv_graph*labels

<a id='related_alerts'></a>[Contents](#toc)
# Related Alerts
For a subset of entities in the alert we can search for any alerts that have that entity in common. Currently this query looks for alerts that share the same Host, Account or Process and lists them below. 
**Notes:**
- Some alert types do not include all of these entity types.
- The original alert will be included in the "Related Alerts" set if it occurs within the query time boundary set below.

The query time boundaries default to a longer period than when searching for the alert. You can extend the time boundary searched before or after the alert time. If the widget doesn't support the time boundary that you want you can change the max_before and max_after parameters in the call to QueryTime below to extend the possible time boundaries.

In [ ]:
# set the origin time to the time of our alert
query_times = mas.QueryTime(units='day', origin_time=security_alert.TimeGenerated, 
                            max_before=28, max_after=1, before=5)
query_times.display()

In [ ]:
related_alerts = qry.list_related_alerts(provs=[query_times, security_alert])

host_alert_items = related_alerts\
    .query('host_match == @True')[['AlertType', 'StartTimeUtc']]\
    .groupby('AlertType').StartTimeUtc.agg('count').to_dict()
acct_alert_items = related_alerts\
    .query('acct_match == @True')[['AlertType', 'StartTimeUtc']]\
    .groupby('AlertType').StartTimeUtc.agg('count').to_dict()
proc_alert_items = related_alerts\
    .query('proc_match == @True')[['AlertType', 'StartTimeUtc']]\
    .groupby('AlertType').StartTimeUtc.agg('count').to_dict()

def print_related_alerts(alertDict, entityType, entityName):
    if len(alertDict) > 0:
        print('Found {} different alert types related to this {} (\'{}\')'.format(len(alertDict), entityType, entityName))
        for (k,v) in alertDict.items():
            print('    {}, Count of alerts: {}'.format(k, v))
    else:
        print('No alerts for {} entity \'{}\''.format(entityType, entityName))

print_related_alerts(host_alert_items, 'host', security_alert.hostname)
print_related_alerts(acct_alert_items, 'account', 
                     security_alert.primary_account.qualified_name if security_alert.primary_account
                     else None)
print_related_alerts(proc_alert_items, 'process', 
                     security_alert.primary_process.ProcessFilePath if security_alert.primary_process
                     else None)
nbdisp.display_timeline(data=related_alerts, source_columns = ['AlertName'], title='Alerts', height=100)

### Show these related alerts on a graph
This should indicate which entities the other alerts are related to.

This can be unreadable with a lot of alerts. Use the matplotlib interactive zoom control to zoom in to part of the graph.

In [ ]:
# Draw a graph of this (add to entity graph)
%matplotlib notebook
%matplotlib inline

rel_alert_graph = mas.add_related_alerts(related_alerts=related_alerts,
                                         alertgraph=alertentity_graph)
nbdisp.draw_alert_entity_graph(rel_alert_graph, width=15)

### Browse List of Related Alerts
Select an Alert to view details. 

If you want to investigate that alert - copy its *SystemAlertId* property and open a new instance of this notebook to investigate this alert.

In [ ]:
related_alerts['CompromisedEntity'] = related_alerts['Computer']

def disp_full_alert(alert):
    global related_alert
    related_alert = mas.SecurityAlert(alert)
    nbdisp.display_alert(related_alert, show_entities=True)

print('Selected alert is available as \'related_alert\' variable.')
rel_alert_select = mas.AlertSelector(alerts=related_alerts, action=disp_full_alert)
rel_alert_select.display()


<a id='processtree'></a>[Contents](#toc)
# Get Process Tree
If the alert has a process entity this section tries to retrieve the entire process tree to which that process belongs.

Notes:
- The alert must have a process entity
- Only processes started within the query time boundary will be included
- Ancestor and descented processes are retrieved to two levels (i.e. the parent and grandparent of the alert process plus any child and grandchild processes).
- Sibling processes are the processes that share the same parent as the alert process
- This can be a long-running query, especially if a wide time window is used! Caveat Emptor!

The source (alert) process is shown in red.

What's shown for each process:
- Each process line is indented according to its position in the tree hierarchy
- Top line fields:
  - \[relationship to source process:lev# - where # is the hops away from the source process\]
  - Process creation date-time (UTC)
  - Process Image path
  - PID - Process Id
  - SubjSess - the session Id of the process spawning the new process
  - TargSess - the new session Id if the process is launched in another context/session. If 0/0x0 then the process is launched in the same session as its parent
- Second line fields:
  - Process command line
  - Account - name of the account context in which the process is running

In [ ]:
# set the origin time to the time of our alert
query_times = mas.QueryTime(units='minute', origin_time=security_alert.origin_time)
query_times.display()

In [ ]:
if security_alert.primary_process and security_alert.primary_process.ProcessId:
    process_tree = qry.get_process_tree(provs=[query_times, security_alert])

    # Print out the text view of the process tree
    nbdisp.display_process_tree(process_tree)
else:
    print('This alert has no process entity. See later in the notebook to retrieve all processes')


<a id='processtimeline'></a>[Contents](#toc)
## Process TimeLine
This shows each process in the process tree on a timeline view.

Labelling of individual process is very performance intensive and often results in nothing being displayed at all! Besides, for large numbers of processes it would likely result in an unreadable mess. 

Your main tools for negotiating the timeline are the Hover tool (toggled on and off by the speech bubble icon) and the wheel-zoom and pan tools (the former is an icon with an elipse and a magnifying glass, the latter is the crossed-arrows icon). The wheel zoom is particularly useful.

As you hover over each process it will display the image name, PID and commandline.

Also shown on the graphic is the timestamp line of the source/alert process.

In [ ]:
# Show timeline of events
nbdisp.display_timeline(data=process_tree, alert=security_alert, title='Alert Process Session', height=250)

<a id='process_clustering'></a>[Contents](#toc)
# Other Processes on Host - Clustering
Sometimes you don't have a source process to work with. Other times it's just useful to see what else is going on on the host. This section retrieves all processes on the host within the time bounds
set in the query times widget.

You can display the raw output of this by looking at the *processes_on_host* dataframe. Just copy this into a new cell and hit Ctrl-Enter.

Usually though, the results return a lot of very repetitive and unintersting system processes so we attempt to cluster these to make the view easier to negotiate. 
To do this we process the raw event list output to extract a few features that render strings (such as commandline)into numerical values. The default below uses the following features:
- commandLineTokensFull - this is a count of common delimiters in the commandline 
  (given by this regex r'[\s\-\\/\.,"\'|&:;%$()]'). The aim of this is to capture the commandline structure while ignoring variations on what is essentially the same pattern (e.g. temporary path GUIDs, target IP or host names, etc.)
- pathScore - this sums the ordinal (character) value of each character in the path (so /bin/bash and /bin/bosh would have similar scores).
- isSystemSession - 1 if this is a root/system session, 0 if anything else.

Then we run a clustering algorithm (DBScan in this case) on the process list. The result groups similar (noisy) processes together and leaves unique process patterns as single-member clusters.

#### Clustered Processes (i.e. processes that have a cluster size > 1)

In [ ]:
from msticpy.sectools.eventcluster import dbcluster_events, add_process_features

processes_on_host = qry.list_processes(provs=[query_times, security_alert])
feature_procs = add_process_features(input_frame=processes_on_host,
                                     path_separator=security_alert.path_separator)


# you might need to play around with the max_cluster_distance parameter.
# decreasing this gives more clusters.
(clus_events, dbcluster, x_data) = dbcluster_events(data=feature_procs,
                                                    cluster_columns=['commandlineTokensFull', 
                                                                     'pathScore', 
                                                                     'isSystemSession'],
                                                    max_cluster_distance=0.0001)
print('Number of input events:', len(feature_procs))
print('Number of clustered events:', len(clus_events))
clus_events[['ClusterSize', 'processName']][clus_events['ClusterSize'] > 1].plot.bar(x='processName', 
                                                                                     title='Process names with Cluster > 1', 
                                                                                     figsize=(12,3));

In [ ]:
# Looking at the variability of commandlines and process image paths
import seaborn as sns
sns.set(style="darkgrid")

proc_plot = sns.catplot(y="processName", x="commandlineTokensFull", 
                        data=feature_procs.sort_values('processName'),
                        kind='box', height=10)
proc_plot.fig.suptitle('Variability of Commandline Tokens', x=1, y=1)

proc_plot = sns.catplot(y="processName", x="pathLogScore", 
                        data=feature_procs.sort_values('processName'),
                        kind='box', height=10, hue='isSystemSession')
proc_plot.fig.suptitle('Variability of Path', x=1, y=1);

The top graph shows that, for a given process, some have a wide variability in their command line content while the majority have little or none. Looking at a couple of examples - like cmd.exe, powershell.exe, reg.exe, net.exe - we can recognize several common command line tools.

The second graph shows processes by full process path content. We wouldn't normally expect to see variation here - as is the cast with most. There is also quite a lot of variance in the score making it a useful proxy feature for unique path name (this means that proc1.exe and proc2.exe that have the same commandline score won't get collapsed into the same cluster).

Any process with a spread of values here means that we are seeing the same process name (but not necessarily the same file) is being run from different locations.

In [ ]:
resp = input('View the clustered data? y/n')
if resp == 'y':
    display(clus_events.sort_values('TimeGenerated')[['TimeGenerated', 'LastEventTime',
                                                      'NewProcessName', 'CommandLine', 
                                                      'ClusterSize', 'commandlineTokensFull',
                                                      'pathScore', 'isSystemSession']])

In [ ]:
# Look at clusters for individual process names
def view_cluster(exe_name):
    display(clus_events[['ClusterSize', 'processName', 'CommandLine', 'ClusterId']][clus_events['processName'] == exe_name])
view_cluster('reg.exe')

In [ ]:
# Show all clustered processes

# Create label with unqualified path
labelled_df = processes_on_host.copy()
labelled_df['label'] = labelled_df.apply(lambda x: x.NewProcessName.split(security_alert.path_separator)[-1], axis=1)

%matplotlib inline
#%matplotlib notebook
plt.rcParams['figure.figsize'] = (15,10)
nbdisp.plot_cluster(dbcluster, labelled_df, x_data, plot_label='label', plot_features=[0,1], verbose=False, cut_off=3,
             xlabel='CmdLine Tokens', ylabel='Path Score');


### Time showing clustered vs. original data

In [ ]:
# Show timeline of events - clustered events
nbdisp.display_timeline(data=clus_events, overlay_data=processes_on_host, 
                         alert=security_alert, title='Distinct Host Processes (top) and All Proceses (bottom)')

<a id='cmdlineiocs'></a>[Contents](#toc)
# Base64 Decode and Check for IOCs
This section looks for Indicators of Compromise (IoC) within the data sets passed to it.

The first section looks at the commandline for the alert process (if any). It also looks for base64 encoded strings within the data - this is a common way of hiding attacker intent. It attempts to decode any strings that look like base64. Additionally, if the base64 decode operation returns any items that look like a base64 encoded string or file, a gzipped binary sequence, a zipped or tar archive, it will attempt to extract the contents before searching for potentially interesting items.

In [ ]:
process = security_alert.primary_process
ioc_extractor = sectools.IoCExtract()

if process:
    # if nothing is decoded this just returns the input string unchanged
    base64_dec_str, _ = sectools.b64.unpack_items(input_string=process["CommandLine"])
    if base64_dec_str and '<decoded' in base64_dec_str:
        print('Base64 encoded items found.')
        print(base64_dec_str)
        
    # any IoCs in the string?
    iocs_found = ioc_extractor.extract(base64_dec_str)
    
    if iocs_found:
        print('\nPotential IoCs found in alert process:')
        display(iocs_found)
else:
    print('Nothing to process')


### If we have a process tree, look for IoCs in the whole data set
You can replace the data=process_tree parameter to ioc_extractor.extract() to pass other data frames.
use the columns parameter to specify which column or columns that you want to search.

In [ ]:
ioc_extractor = sectools.IoCExtract()
ioc_df = ioc_extractor.extract(data=process_tree, columns=['CommandLine'], os_family=security_alert.os_family)
if len(ioc_df):
    display(HTML("<h3>IoC patterns found in process tree.</h3>"))
    display(ioc_df)

### If any Base64 encoded strings, decode and search for IoCs in the results.
For simple strings the Base64 decoded output is straightforward. However for nested encodings this can get a little complex and difficult to represent in a tabular format.

**Columns**
 - reference - The index of the row item in dotted notation in depth.seq pairs (e.g. 1.2.2.3 would be the 3 item at depth 3 that is a child of the 2nd item found at depth 1). This may not always be an accurate notation - it is mainly use to allow you to associate an individual row with the reference value contained in the full_decoded_string column of the topmost item).
 - original_string - the original string before decoding.
 - file_name - filename, if any (only if this is an item in zip or tar file).
 - file_type - a guess at the file type (this is currently elementary and only includes a few file types).
 - input_bytes - the decoded bytes as a Python bytes string.
 - decoded_string - the decoded string if it can be decoded as a UTF-8 or UTF-16 string. Note: binary sequences may often successfully decode as UTF-16 strings but, in these cases, the decodings are meaningless.
 - encoding_type - encoding type (UTF-8 or UTF-16) if a decoding was possible, otherwise 'binary'.
 - file_hashes - collection of file hashes for any decoded item.
 - md5 - md5 hash as a separate column.
 - sha1 - sha1 hash as a separate column.
 - sha256 - sha256 hash as a separate column.
 - printable_bytes - printable version of input_bytes as a string of \xNN values
 - src_index - the index of the row in the input dataframe from which the data came.
 - full_decoded_string - the full decoded string with any decoded replacements. This is only really useful for top-level items, since nested items will only show the 'full' string representing the child fragment.

In [ ]:
dec_df = sectools.b64.unpack_items(data=process_tree, column='CommandLine')
if len(dec_df) > 0:
    display(HTML("<h3>Decoded base 64 command lines</h3>"))
    display(HTML("Warning - some binary patterns may be decodable as unicode strings"))
    display(dec_df[['full_decoded_string', 'original_string', 'decoded_string', 'input_bytes', 'file_hashes']])

    ioc_dec_df = ioc_extractor.extract(data=dec_df, columns=['full_decoded_string'])
    if len(ioc_dec_df):
        display(HTML("<h3>IoC patterns found in base 64 decoded data</h3>"))
        display(ioc_dec_df)
        ioc_df = ioc_df.append(ioc_dec_df ,ignore_index=True)
else:
    print("No base64 encodings found.")

<a id='virustotallookup'></a>[Contents](#toc)
## Virus Total Lookup
This section uses the popular Virus Total service to check any recovered IoCs against VTs database.

To use this you need an API key from virus total, which you can obtain here: https://www.virustotal.com/.

Note that VT throttles requests for free API keys to 4/minute. If you are unable to process the entire data set, try splitting it and submitting smaller chunks.

**Things to note:**
- Virus Total lookups include file hashes, domains, IP addresses and URLs.
- The returned data is slightly different depending on the input type
- The VTLookup class tries to screen input data to prevent pointless lookups. E.g.:
  - Only public IP Addresses will be submitted (no loopback, private address space, etc.)
  - URLs with only local (unqualified) host parts will not be submitted.
  - Domain names that are unqualified will not be submitted.
  - Hash-like strings (e.g 'AAAAAAAAAAAAAAAAAA') that do not appear to have enough entropy to be a hash will not be submitted.

**Output Columns**
 - Observable - The IoC observable submitted
 - IoCType - the IoC type
 - Status - the status of the submission request
 - ResponseCode - the VT response code
 - RawResponse - the entire raw json response
 - Resource - VT Resource
 - SourceIndex - The index of the Observable in the source DataFrame. You can use this to rejoin to your original data.
 - VerboseMsg - VT Verbose Message
 - ScanId - VT Scan ID if any
 - Permalink - VT Permanent URL describing the resource
 - Positives - If this is not zero, it indicates the number of malicious reports that VT holds for this observable.
 - MD5 - The MD5 hash, if any
 - SHA1 - The MD5 hash, if any
 - SHA256 - The MD5 hash, if any
 - ResolvedDomains - In the case of IP Addresses, this contains a list of all domains that resolve to this IP address
 - ResolvedIPs - In the case Domains, this contains a list of all IP addresses resolved from the domain.
 - DetectedUrls - Any malicious URLs associated with the observable.

In [ ]:
vt_key = mas.GetEnvironmentKey(env_var='VT_API_KEY',
                           help_str='To obtain an API key sign up here https://www.virustotal.com/',
                           prompt='Virus Total API key:')
vt_key.display()

In [ ]:
if vt_key.value:
    vt_lookup = sectools.VTLookup(vt_key.value, verbosity=2)

    print(f'{len(ioc_df)} items in input frame')
    supported_counts = {}
    for ioc_type in vt_lookup.supported_ioc_types:
        supported_counts[ioc_type] = len(ioc_df[ioc_df['IoCType'] == ioc_type])
    print('Items in each category to be submitted to VirusTotal')
    print('(Note: items have pre-filtering to remove obvious erroneous '
          'data and false positives, such as private IPaddresses)')
    print(supported_counts)
    print('-' * 80)
    vt_results = vt_lookup.lookup_iocs(data=ioc_df, type_col='IoCType', src_col='Observable')
    
    pos_vt_results = vt_results.query('Positives > 0')
    if len(pos_vt_results) > 0:
        display(HTML(f'<h3>{len(pos_vt_results)} Positive Results Found</h3>'))
        display(pos_vt_results[['Observable', 'IoCType','Permalink', 
                                'ResolvedDomains', 'ResolvedIPs', 
                                'DetectedUrls', 'RawResponse']])
    display(HTML('<h3>Other results</h3>'))
    display(vt_results.query('Status == \'Success\''))

To view the raw response for a specific row.
```
import json
row_idx = 0 # The row number from one of the above dataframes
raw_response = json.loads(pos_vt_results['RawResponse'].loc[row_idx])
raw_response
```

<a id='cmdlineonotherhosts'></a>[Contents](#toc)
# Alert command line - Occurrence on other hosts in workspace
To get a sense of whether the alert process is something that is occuring on other hosts, run this section.

This might tell you that the alerted process is actually a commonly-run process and the alert is a false positive. Alternatively, it may tell you that a real infection or attack is happening on other hosts in your environment.

In [ ]:
# set the origin time to the time of our alert
query_times = mas.QueryTime(units='day', before=5, max_before=20,
                            after=1, max_after=10,
                            origin_time=security_alert.origin_time)
query_times.display()

In [ ]:
# API ILLUSTRATION - Find the query to use
qry.list_queries()

In [ ]:
# API ILLUSTRATION - What does the query look like?
qry.query_help('list_hosts_matching_commandline')

In [ ]:
# This query needs a commandline parameter which isn't supplied
# by default from the the alert 
# - so extract and escape this from the process
commandline = security_alert.primary_process.CommandLine
commandline = mas.utility.escape_windows_path(commandline)
#commandline = commandline.replace('\'', '\\\'')
proc_match_in_ws = qry.list_hosts_matching_commandline(provs=[query_times, security_alert],
                                                                commandline=commandline)

# Check the results
if proc_match_in_ws is None or len(proc_match_in_ws) == 0:
    print('No proceses with matching commandline found in on other hosts in workspace')
    print('between', query_times.start, 'and', query_times.end)
else:
    hosts = proc_match_in_ws['Computer'].drop_duplicates().shape[0]
    processes = proc_match_in_ws.shape[0]
    print('{numprocesses} proceses with matching commandline found on {numhosts} hosts in workspace'\
         .format(numprocesses=processes, numhosts=hosts))
    print('between', query_times.start, 'and', query_times.end)
    print('To examine these execute the dataframe \'{}\' in a new cell'.format('proc_match_in_ws'))
    print(proc_match_in_ws[['TimeCreatedUtc','Computer', 'NewProcessName', 'CommandLine']].head())
    

<a id='host_logons'></a>[Contents](#toc)
# Host Logons
This section retrieves the logon events on the host in the alert.

You may want to use the query times to search over a broader range than the default.

In [ ]:
# set the origin time to the time of our alert
query_times = mas.QueryTime(units='day', origin_time=security_alert.origin_time,
                           before=5, after=0, max_before=20, max_after=1)
query_times.display()

<a id='logonaccount'></a>[Contents](#toc)
## Alert Logon Account
The logon associated with the process in the alert.

In [ ]:
if security_alert.primary_account:
    logon_event = qry.get_host_logon(provs=[query_times, security_alert])
    nbdisp.display_logon_data(logon_event, security_alert)
else:
    print('No account entity in the source alert.')

### All Host Logons
Since the number of logon events may be large and, in the case of system logons, very repetitive, we use clustering to try to identity logons with unique characteristics.

In this case we use the numeric score of the account name and the logon type (i.e. interactive, service, etc.). The results of the clustered logons are shown below along with a more detailed, readable printout of the logon event information. The data here will vary depending on whether this is a Windows or Linux host.

In [ ]:
from msticpy.sectools.eventcluster import dbcluster_events, add_process_features, _string_score

host_logons = qry.list_host_logons(provs=[query_times, security_alert])
if len(host_logons) > 0:
    logon_features = host_logons.copy()
    logon_features['AccountNum'] = host_logons.apply(lambda x: _string_score(x.Account), axis=1)
    logon_features['LogonHour'] = host_logons.apply(lambda x: x.TimeGenerated.hour, axis=1)

    # you might need to play around with the max_cluster_distance parameter.
    # decreasing this gives more clusters.
    (clus_logons, _, _) = dbcluster_events(data=logon_features, time_column='TimeGenerated',
                                           cluster_columns=['AccountNum',
                                                            'LogonType'],
                                                             max_cluster_distance=0.0001)
    print('Number of input events:', len(host_logons))
    print('Number of clustered events:', len(clus_logons))
    print('\nDistinct host logon patterns:')
    display(clus_logons.sort_values('TimeGenerated'))
else:
    print('No logon events found for host.')

In [ ]:
# Display logon details
nbdisp.display_logon_data(clus_logons, security_alert)

### Comparing All Logons with Clustered results relative to Alert time line

In [ ]:
# Show timeline of events - all logons + clustered logons
nbdisp.display_timeline(data=host_logons, overlay_data=clus_logons,
                         alert=security_alert, 
                         source_columns=['Account', 'LogonType'],
                         title='All Host Logons')

### View Process Session and Logon Events in Timelines
This shows the timeline of the clustered logon events with the process tree obtained earlier. This allows you to get a sense of which logon was responsible for the process tree session whether any additional logons (e.g. creating a process as another user) might be associated with the alert timeline.

*Note you should use the pan and zoom tools to align the timelines since the data may be over different time ranges.*

In [ ]:
# Show timeline of events - all events
nbdisp.display_timeline(data=clus_logons, source_columns=['Account', 'LogonType'],
                         alert=security_alert,
                         title='Clustered Host Logons', height=200)
nbdisp.display_timeline(data=process_tree, alert=security_alert, title='Alert Process Session', height=200)

In [ ]:
# Counts of Logon types by Account
host_logons[['Account', 'LogonType', 'TimeGenerated']].groupby(['Account','LogonType']).count()

<a id='failed logons'></a>[Contents](#toc)
## Failed Logons

In [ ]:
failedLogons = qry.list_host_logon_failures(provs=[query_times, security_alert])
if failedLogons.shape[0] == 0:
    display(print('No logon failures recorded for this host between {security_alert.start} and {security_alert.start}'))

failedLogons

<a id='appendices'></a>[Contents](#toc)
# Appendices

## Available DataFrames

In [ ]:
print('List of current DataFrames in Notebook')
print('-' * 50)
current_vars = list(locals().keys())
for var_name in current_vars:
    if isinstance(locals()[var_name], pd.DataFrame) and not var_name.startswith('_'):
        print(var_name)

## Saving Data to CSV
To save the contents of a pandas DataFrame to an CSV
use the following syntax
```
host_logons.to_csv('host_logons.csv')
```

## Saving Data to Excel
To save the contents of a pandas DataFrame to an Excel spreadsheet
use the following syntax
```
writer = pd.ExcelWriter('myWorksheet.xlsx')
my_data_frame.to_excel(writer,'Sheet1')
writer.save()
```